In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_csv('features_for_fit10_ufix.csv')
df

,MSA,Employment,GDP,Population,Unemployment
0,"Abilene, TX",10.945263,32.378757,3.910378,3.0
1,"Akron, OH",7.761940,30.070473,0.063724,4.3
2,"Albany, GA",7.061047,29.231972,-4.813001,4.3
3,"Albany-Lebanon, OR",15.685181,52.685485,10.999991,4.3
4,"Albany-Schenectady-Troy, NY",9.664928,39.598832,1.067523,3.7
...,...,...,...,...,...
378,"Yakima, WA",14.634453,47.539475,2.709043,7.0
379,"York-Hanover, PA",9.534342,24.927250,3.133806,3.8
380,"Youngstown-Warren-Boardman, OH-PA",0.967322,18.783134,-5.089178,5.7
381,"Yuba City, CA",20.879314,36.901149,5.110743,6.8


In [3]:
print(df['Employment'].mean())
print(df['GDP'].mean())
print(df['Unemployment'].mean())

13.716340750885117
37.50540560933163
3.8827676240208877


In [4]:
# Target "Emerging" and "Static" based off of top 40 population ROC
df.loc[df['Population'] > 15.6, 'Target'] = "Emerging"
df.loc[df['Population'] < 15.6, 'Target'] = "Static"
df

,MSA,Employment,GDP,Population,Unemployment,Target
0,"Abilene, TX",10.945263,32.378757,3.910378,3.0,Static
1,"Akron, OH",7.761940,30.070473,0.063724,4.3,Static
2,"Albany, GA",7.061047,29.231972,-4.813001,4.3,Static
3,"Albany-Lebanon, OR",15.685181,52.685485,10.999991,4.3,Static
4,"Albany-Schenectady-Troy, NY",9.664928,39.598832,1.067523,3.7,Static
...,...,...,...,...,...,...
378,"Yakima, WA",14.634453,47.539475,2.709043,7.0,Static
379,"York-Hanover, PA",9.534342,24.927250,3.133806,3.8,Static
380,"Youngstown-Warren-Boardman, OH-PA",0.967322,18.783134,-5.089178,5.7,Static
381,"Yuba City, CA",20.879314,36.901149,5.110743,6.8,Static


In [5]:
df.loc[df['Target']=='Emerging']

,MSA,Employment,GDP,Population,Unemployment,Target
19,"Auburn-Opelika, AL",31.334402,55.085479,16.867196,2.7,Emerging
21,"Austin-Round Rock-Georgetown, TX",44.714842,76.983055,28.919272,2.7,Emerging
32,"Bend, OR",41.152886,99.963782,25.326166,3.9,Emerging
36,"Bismarck, ND",14.407954,52.376556,16.053172,2.6,Emerging
41,"Boise City, ID",32.991115,62.718127,21.256389,2.7,Emerging
54,"Cape Coral-Fort Myers, FL",41.614405,56.665186,24.196670,3.1,Emerging
63,"Charleston-North Charleston, SC",33.679292,66.307795,20.177092,2.4,Emerging
64,"Charlotte-Concord-Gastonia, NC-SC",33.285144,62.447199,17.188498,3.5,Emerging
74,"Coeur d'Alene, ID",25.377724,52.481940,19.336973,3.6,Emerging
84,"Crestview-Fort Walton Beach-Destin, FL",27.494426,54.519299,20.716562,2.7,Emerging


In [6]:
y = df['Target']
X = df.drop(columns=['MSA', 'Population', 'Target'])
X

,Employment,GDP,Unemployment
0,10.945263,32.378757,3.0
1,7.761940,30.070473,4.3
2,7.061047,29.231972,4.3
3,15.685181,52.685485,4.3
4,9.664928,39.598832,3.7
...,...,...,...
378,14.634453,47.539475,7.0
379,9.534342,24.927250,3.8
380,0.967322,18.783134,5.7
381,20.879314,36.901149,6.8


In [7]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
Counter(y_train)

Counter({'Static': 255, 'Emerging': 32})

In [8]:
# instantiate the model a fit using the training sets
model = LogisticRegression(solver='lbfgs', max_iter=50, random_state=42, class_weight={'Emerging':.5})
model.fit(X_train, y_train)

LogisticRegression(class_weight={'Emerging': 0.5}, max_iter=50, random_state=42)

In [9]:
# test the model accuracy
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9375
[[ 4  4]
 [ 2 86]]
              precision    recall  f1-score   support

    Emerging       0.67      0.50      0.57         8
      Static       0.96      0.98      0.97        88

    accuracy                           0.94        96
   macro avg       0.81      0.74      0.77        96
weighted avg       0.93      0.94      0.93        96



In [10]:
# import feature set to run predictions on 2024 data
df_2024 = pd.read_csv('2024_ROC_rank_total.csv')
df_2024.head()

,CBSA,Metropolitan_Area,2024_Pop_ROC,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC,Pop_ROC_Rank,Unem_ROC_Rank,Emp_ROC_Rank,GDP_ROC_Rank,Rank_Total
0,10180,"Abilene, TX",2.0,1.15,3.040151,8.072256,217.5,38.0,134.0,297.0,686.5
1,10500,"Albany, GA",-3.2,2.58,0.089023,7.199080,370.0,349.0,260.0,312.0,1291.0
2,10420,"Akron, OH",0.2,1.20,1.238754,12.475028,286.5,76.5,214.0,201.0,778.0
3,10540,"Albany-Lebanon, OR",3.9,2.05,-1.245487,17.059092,142.0,304.5,312.0,88.0,846.5
4,10580,"Albany-Schenectady-Troy, NY",1.0,1.17,1.270928,14.602271,257.5,53.0,211.0,155.0,676.5


In [11]:
print(df_2024['2024_Emp_ROC'].mean())
print(df_2024['2024_GDP_ROC'].mean())
print(df_2024['2024_Unem_ROC'].mean())

1.8355239515649788
12.498457627383495
1.6673368146214056


In [12]:
df_2024.columns.tolist()

['CBSA',
 'Metropolitan_Area',
 '2024_Pop_ROC',
 '2024_Unem_ROC',
 '2024_Emp_ROC',
 '2024_GDP_ROC',
 'Pop_ROC_Rank',
 'Unem_ROC_Rank',
 'Emp_ROC_Rank',
 'GDP_ROC_Rank',
 'Rank_Total']

In [13]:
# set x2 data for logisticRegression prediction
X2 = df_2024.drop(columns=['CBSA',
 'Metropolitan_Area',
 'Pop_ROC_Rank',
 'Unem_ROC_Rank',
 '2024_Pop_ROC',
 'Emp_ROC_Rank',
 'GDP_ROC_Rank',
 'Rank_Total'])
X2.head()

,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC
0,1.15,3.040151,8.072256
1,2.58,0.089023,7.199080
2,1.20,1.238754,12.475028
3,2.05,-1.245487,17.059092
4,1.17,1.270928,14.602271


In [14]:
# reorder columns to match training data columns
X2 = X2[['2024_Emp_ROC','2024_GDP_ROC','2024_Unem_ROC']]
X2.head()

,2024_Emp_ROC,2024_GDP_ROC,2024_Unem_ROC
0,3.040151,8.072256,1.15
1,0.089023,7.199080,2.58
2,1.238754,12.475028,1.20
3,-1.245487,17.059092,2.05
4,1.270928,14.602271,1.17


In [15]:
# run the logisticRegression model over our 2024 data set
emerging_pred = model.predict(X2)
# added predicted classifiers back into original dataframe
df_2024['Predicted_Classifier_LR'] = emerging_pred
# dispay MSA that our ML model predicted to be emerging in 2024
df_2024.loc[df_2024['Predicted_Classifier_LR']=='Emerging']

,CBSA,Metropolitan_Area,2024_Pop_ROC,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC,Pop_ROC_Rank,Unem_ROC_Rank,Emp_ROC_Rank,GDP_ROC_Rank,Rank_Total,Predicted_Classifier_LR


In [16]:
# use svm to predict emerging
# create SVM classifier
model_svm = SVC(kernel='linear')
model_svm.fit(X_train, y_train)
emerging_pred_svm = model_svm.predict(X2)
df_2024['Predicted_Classifier_svm'] = emerging_pred_svm
df_2024.loc[df_2024['Predicted_Classifier_svm']=='Emerging']

,CBSA,Metropolitan_Area,2024_Pop_ROC,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC,Pop_ROC_Rank,Unem_ROC_Rank,Emp_ROC_Rank,GDP_ROC_Rank,Rank_Total,Predicted_Classifier_LR,Predicted_Classifier_svm


In [17]:
# Creating a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=42, class_weight={'Emerging':1.75})
rf_model.fit(X_train, y_train)
forest_predict = rf_model.predict(X2)
df_2024['Predicted_Classifier_RFC'] = forest_predict
df_2024.loc[df_2024['Predicted_Classifier_RFC']=='Emerging']

,CBSA,Metropolitan_Area,2024_Pop_ROC,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC,Pop_ROC_Rank,Unem_ROC_Rank,Emp_ROC_Rank,GDP_ROC_Rank,Rank_Total,Predicted_Classifier_LR,Predicted_Classifier_svm,Predicted_Classifier_RFC


In [18]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.25,
                                        max_features=3,
                                        max_depth=3,
                                        random_state=42)

# Fit the model
classifier.fit(X_train, y_train)

# Make Prediction
predictions = classifier.predict(X2)
df_2024['Predicted_Classifier_GBC'] = forest_predict
df_2024.loc[df_2024['Predicted_Classifier_GBC']=='Emerging']

,CBSA,Metropolitan_Area,2024_Pop_ROC,2024_Unem_ROC,2024_Emp_ROC,2024_GDP_ROC,Pop_ROC_Rank,Unem_ROC_Rank,Emp_ROC_Rank,GDP_ROC_Rank,Rank_Total,Predicted_Classifier_LR,Predicted_Classifier_svm,Predicted_Classifier_RFC,Predicted_Classifier_GBC
